In [1]:
import numpy as np
from serpentGenerator.functions.pin import pin
from serpentGenerator.data.materialLibrary import MATLIB
from serpentGenerator.functions.material import material
from serpentGenerator.functions.sqLattice import sqLat
from serpentGenerator.functions.hexLattice import hexLat
from serpentGenerator.functions.pinStack import pinStack
from serpentGenerator.functions.surf import surf
from serpentGenerator.functions.lats import lats
from serpentGenerator.functions.mats import mats
from serpentGenerator.functions.pins import pins
from serpentGenerator.functions.core import core
from serpentGenerator.functions.housing import housing

In [2]:
fuel = MATLIB['UO2'].duplicateMat("fuel")
fuel.set('fractions', np.array([-0.04319337211, -0.83830401789, -.11850261])) # 4.9 % wt enrichment

mod = MATLIB['lightWater'].duplicateMat("water")
mod.set("isBurn", False)

clad = MATLIB['Zr'].duplicateMat("clad")
clad.set("isBurn", False)

ref = MATLIB['plenum'].duplicateMat("reflector") # reflector plenum
ref.set("isBurn", False)

matList = mats()
matList.addMats([fuel, mod, clad, ref])

In [3]:
F = pin("F", 3)
F.set('materials', np.array([fuel, mod, clad]))
F.set('radii', np.array([.4095, 4750]))

G = pin("G", 3)
G.set('materials', np.array([mod, clad, mod]))
G.set('radii', np.array([.5715, 6120]))

W = pin("W", 3)
W.set('materials', np.array([mod]))

RT = pin("RT", 1) #top reflector
RT.set('materials', np.array([ref]))

RB =  RT.duplicatePin("RB")

pinList = pins()
pinList.addPins([F, G, W, RT, RB])

In [4]:
FC = pinStack("FC", 0, 0, 3)
FC.setStack(np.array([RB, F, RT]), np.array([-20.00, 200, 220]))

GT = pinStack("GT", 0, 0, 3)
GT.setStack(np.array([RB, G, RT]), np.array([-20.00, 200, 220]))

WG = pinStack("WG", 0, 0, 3)
WG.setStack(np.array([RB, W, RT]), np.array([-20.00, 200, 220]))

latList = lats()
latList.addLats([FC, GT, WG])

In [5]:
FA = sqLat("FA", 0, 0, 19, 1.260)

FAMap = np.array([
    [WG, WG, WG, WG, WG, WG, WG, WG, WG, WG, WG, WG, WG, WG, WG, WG, WG, WG, WG],
    [WG, FC, FC, FC, FC, FC, FC, FC, FC, FC, FC, FC, FC, FC, FC, FC, FC, FC, WG],
    [WG, FC, FC, FC, FC, FC, FC, FC, FC, FC, FC, FC, FC, FC, FC, FC, FC, FC, WG],
    [WG, FC, FC, FC, FC, FC, GT, FC, FC, GT, FC, FC, GT, FC, FC, FC, FC, FC, WG],
    [WG, FC, FC, FC, GT, FC, FC, FC, FC, FC, FC, FC, FC, FC, GT, FC, FC, FC, WG],
    [WG, FC, FC, FC, FC, FC, FC, FC, FC, FC, FC, FC, FC, FC, FC, FC, FC, FC, WG],
    [WG, FC, FC, GT, FC, FC, GT, FC, FC, GT, FC, FC, GT, FC, FC, GT, FC, FC, WG],
    [WG, FC, FC, FC, FC, FC, FC, FC, FC, FC, FC, FC, FC, FC, FC, FC, FC, FC, WG],
    [WG, FC, FC, FC, FC, FC, FC, FC, FC, FC, FC, FC, FC, FC, FC, FC, FC, FC, WG],
    [WG, FC, FC, GT, FC, FC, GT, FC, FC, GT, FC, FC, GT, FC, FC, GT, FC, FC, WG],
    [WG, FC, FC, FC, FC, FC, FC, FC, FC, FC, FC, FC, FC, FC, FC, FC, FC, FC, WG],
    [WG, FC, FC, FC, FC, FC, FC, FC, FC, FC, FC, FC, FC, FC, FC, FC, FC, FC, WG],
    [WG, FC, FC, GT, FC, FC, GT, FC, FC, GT, FC, FC, GT, FC, FC, GT, FC, FC, WG],
    [WG, FC, FC, FC, FC, FC, FC, FC, FC, FC, FC, FC, FC, FC, FC, FC, FC, FC, WG],
    [WG, FC, FC, FC, GT, FC, FC, FC, FC, FC, FC, FC, FC, FC, GT, FC, FC, FC, WG],
    [WG, FC, FC, FC, FC, FC, GT, FC, FC, GT, FC, FC, GT, FC, FC, FC, FC, FC, WG],
    [WG, FC, FC, FC, FC, FC, FC, FC, FC, FC, FC, FC, FC, FC, FC, FC, FC, FC, WG],
    [WG, FC, FC, FC, FC, FC, FC, FC, FC, FC, FC, FC, FC, FC, FC, FC, FC, FC, WG],
    [WG, WG, WG, WG, WG, WG, WG, WG, WG, WG, WG, WG, WG, WG, WG, WG, WG, WG, WG],
])

FA.setMap(FAMap)


In [6]:
WA = sqLat("WA", 0, 0, 19, 1.260)

WAMap = np.array([
    [WG, WG, WG, WG, WG, WG, WG, WG, WG, WG, WG, WG, WG, WG, WG, WG, WG, WG, WG],
    [WG, WG, WG, WG, WG, WG, WG, WG, WG, WG, WG, WG, WG, WG, WG, WG, WG, WG, WG],
    [WG, WG, WG, WG, WG, WG, WG, WG, WG, WG, WG, WG, WG, WG, WG, WG, WG, WG, WG],
    [WG, WG, WG, WG, WG, WG, WG, WG, WG, WG, WG, WG, WG, WG, WG, WG, WG, WG, WG],
    [WG, WG, WG, WG, WG, WG, WG, WG, WG, WG, WG, WG, WG, WG, WG, WG, WG, WG, WG],
    [WG, WG, WG, WG, WG, WG, WG, WG, WG, WG, WG, WG, WG, WG, WG, WG, WG, WG, WG],
    [WG, WG, WG, WG, WG, WG, WG, WG, WG, WG, WG, WG, WG, WG, WG, WG, WG, WG, WG],
    [WG, WG, WG, WG, WG, WG, WG, WG, WG, WG, WG, WG, WG, WG, WG, WG, WG, WG, WG],
    [WG, WG, WG, WG, WG, WG, WG, WG, WG, WG, WG, WG, WG, WG, WG, WG, WG, WG, WG],
    [WG, WG, WG, WG, WG, WG, WG, WG, WG, WG, WG, WG, WG, WG, WG, WG, WG, WG, WG],
    [WG, WG, WG, WG, WG, WG, WG, WG, WG, WG, WG, WG, WG, WG, WG, WG, WG, WG, WG],
    [WG, WG, WG, WG, WG, WG, WG, WG, WG, WG, WG, WG, WG, WG, WG, WG, WG, WG, WG],
    [WG, WG, WG, WG, WG, WG, WG, WG, WG, WG, WG, WG, WG, WG, WG, WG, WG, WG, WG],
    [WG, WG, WG, WG, WG, WG, WG, WG, WG, WG, WG, WG, WG, WG, WG, WG, WG, WG, WG],
    [WG, WG, WG, WG, WG, WG, WG, WG, WG, WG, WG, WG, WG, WG, WG, WG, WG, WG, WG],
    [WG, WG, WG, WG, WG, WG, WG, WG, WG, WG, WG, WG, WG, WG, WG, WG, WG, WG, WG],
    [WG, WG, WG, WG, WG, WG, WG, WG, WG, WG, WG, WG, WG, WG, WG, WG, WG, WG, WG],
    [WG, WG, WG, WG, WG, WG, WG, WG, WG, WG, WG, WG, WG, WG, WG, WG, WG, WG, WG],
    [WG, WG, WG, WG, WG, WG, WG, WG, WG, WG, WG, WG, WG, WG, WG, WG, WG, WG, WG],
])

WA.setMap(WAMap)

In [12]:
CORE = sqLat("CORE1", 0, 0, 7, 21.50364)
COREMap = np.array([
    [WA, WA, FA, FA, FA, WA, WA],
    [WA, FA, FA, FA, FA, FA, WA],
    [FA, FA, FA, FA, FA, FA, FA],
    [FA, FA, FA, FA, FA, FA, FA],
    [FA, FA, FA, FA, FA, FA, FA],
    [WA, FA, FA, FA, FA, FA, WA],
    [WA, WA, FA, FA, FA, WA, WA],
])

CORE.setMap(COREMap)

print(CORE.toString())

lat CORE1 1 0 0  7 7 21.50364
WA WA FA FA FA WA WA 
WA FA FA FA FA FA WA 
FA FA FA FA FA FA FA 
FA FA FA FA FA FA FA 
FA FA FA FA FA FA FA 
WA FA FA FA FA FA WA 
WA WA FA FA FA WA WA 




In [13]:
width = 141*2
height  = 220
radii = [0,20]
mats = [clad]

h1 = housing(coreWidth=width, coreHeight=height, defaultCRFlag=True, radiiCR= radii, matsCR= mats)

print(h1.toString())

surf cr1 cyl 0.0 0.0 141.0 
surf cr2 cyl 0.0 0.0 161.0 

cell cc1 housing  clad cr1 -cr2 




In [14]:
input = core(mainUniv=CORE, housing=h1, lats=latList, pins=pinList, materials=matList)

input.writeFile("bigT.txt")

text_file = open("bigT.txt") #Display contents of created inputFile
file_content = text_file.read()
print(file_content)
text_file.close()

mat fuel    -10.2141  burn 1  rgb 255 165   0
92235.09c	-0.04319337211
92238.09c	-0.83830401789
8016.09c	-0.11850261

mat water    -0.7 moder ltwr 1001   rgb   0   0 255
1001.06c	0.66667
8016.06c	0.33333

mat clad    -6.55    rgb 128 128 128
40090.06c	0.5145
40091.06c	0.1122
40092.06c	0.1715
40094.06c	0.1738
40096.06c	0.028

mat reflector    0.070462    rgb 255 192 203
40090.06c	0.0059311
40091.06c	0.0012934
40092.06c	0.001977
40094.06c	0.0020035
40096.06c	0.00032278
26054.06c	0.011217
26056.06c	0.17738
26057.06c	0.0042545
26058.06c	0.00054149
24050.06c	0.0024673
24052.06c	0.04758
24053.06c	0.0053951
24054.06c	0.001343
28058.06c	0.017122
28060.06c	0.0065978
28061.06c	0.00028421
28062.06c	0.00091398
28064.06c	0.0002329
25055.06c	0.0056687
1001.06c	0.47165
8016.06c	0.23583

pin F
fuel	0.4095
water	4750.0
clad

pin G
water	0.5715
clad	6120.0
water

pin W
water

pin RT
reflector

pin RB
reflector

lat FC 9 0 0 3
-20.0	RB
200.0	F
220.0	RT

lat GT 9 0 0 3
-20.0	RB
200.0	G
220.0	RT

lat WG 9 